<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/RAG_RespondendoPergunta_Llama2_13B_4bit_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Respondendo Perguntas sobre Textos Longos usando Llama v2.0, Longchain, Chroma e Transformers by HuggingFace

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

**Lista dos modelos:**

https://huggingface.co/models


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Biblioteca para manipular pdf

https://pypi.org/project/pypdf/

In [ ]:
!pip install pypdf==3.16.4

Bibliota de dependência para manipular os embeddings pelo Langchain.

https://pypi.org/project/sentence-transformers/

In [ ]:
!pip install sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Biblioteca que persiste os embeddings e realiza busca semântica.

https://pypi.org/project/chromadb/

In [ ]:
!pip install chromadb==0.4.15

Biblioteca para realizar a divisão por token.

In [ ]:
!pip install tiktoken==0.5.1

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

https://pypi.org/project/langchain/

In [ ]:
!pip install langchain==0.0.323

Dependências do xformers

https://pypi.org/project/lmdb/

In [ ]:
!pip install lmdb==1.4.1
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2 torchtext==0.15.2+cpu torchdata==0.6.1 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xformers 0.0.22.post7 requires torch==2.1.0, but you have torch 2.0.1+cu118 which is incompatible.


Permite maior velocidade e menor consumo de memória nos transformers.

https://pypi.org/project/xformers/

In [ ]:
!pip install xformers==0.0.22.post7

  Using cached torch-2.1.0-cp310-cp310-manylinux1_x86_64.whl (670.2 MB)
  Using cached triton-2.1.0-0-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (89.2 MB)
  Attempting uninstall: triton
    Found existing installation: triton 2.0.0
    Uninstalling triton-2.0.0:
      Successfully uninstalled triton-2.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.15.2+cpu requires torch==2.0.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have to

O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

https://huggingface.co/blog/4bit-transformers-bitsandbytes

https://pypi.org/project/bitsandbytes/

In [ ]:
!pip install bitsandbytes==0.41.1

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.24.0

A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
# Instala a última versão da biblioteca
# !pip install transformers

# A última versão do huggingface apresenta um problema:
# UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1`
# https://discuss.huggingface.co/t/help-with-llama-2-finetuning-setup/50035
# Usar a versão 4.31.0

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.31.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
!pip install huggingface_hub==0.18.0

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.

In [ ]:
from huggingface_hub.hf_api import HfFolder

ACCESS_TOKEN  = 'COLOQUE O SEU TOKEN DE ACESSO AQUI'

HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do LLM

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
# Roda com 4 bits e 8 bits
#nome_modelo_llm = "meta-llama/Llama-2-7b-hf"
# nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

# Roda com 4 bits
#nome_modelo = "meta-llama/Llama-2-13b-hf"
nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador do LLM

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-13b-chat-hf da comunidade...


## 2.4 - Carregando o Modelo LLM

Carregando o **modelo** da comunidade Huggingface.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import time
import torch

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
   bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
   bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
   bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
)

# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-13b-chat-hf da comunidade...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Tempo de carregamento do modelo LLM:  0:02:24 (h:mm:ss)


Carregamento LLama 2 com 8 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import AutoModelForCausalLM
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True,   # Carrega de um repositório confiável
#                                              load_in_8bit=True,
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# print("Tempo de carregamento do modelo:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

In [ ]:
print(model.config.max_position_embeddings)

4096


In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-13b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 13824,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 40,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false
  },
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.31.0",
  "use_cache": true,
  "vocab_size": 32000
}



## 2.5 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import GenerationConfig, pipeline

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)
print("Antes:",generation_config)
generation_config.max_new_tokens = 4096 #Preenche até um comprimento máximo especificado com o argumento max_lengthou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
generation_config.temperature = 0.2 # A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
generation_config.top_p = 0.75 # Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
generation_config.top_k = 5  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
print("Depois:",generation_config)

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # Langchain espera o texto completo
    generation_config=generation_config,
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
)

Antes: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.31.0"
}

Depois: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 4096,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.2,
  "top_k": 5,
  "top_p": 0.75,
  "transformers_version": "4.31.0"
}



In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


## 2.6 - Nome do LM BERT

In [ ]:
# Modelos em Inglês
#nome_modelo_bert = "bert-large-cased"
#nome_modelo_bert = "bert-base-cased"

# Modelos em Português
nome_modelo_bert = "neuralmind/bert-large-portuguese-cased"
#nome_modelo_bert = "neuralmind/bert-base-portuguese-cased"

## 2.7 - Carregando o LM BERT

A classe HuggingFaceBgeEmbeddings realiza o download do BERT via HuggingFace.

Os modelos [BGE](https://python.langchain.com/docs/integrations/text_embedding/bge_huggingface) no HuggingFace são os melhores modelos de embeddings de código aberto. O modelo BGE é criado pela Academia de Inteligência Artificial de Pequim(*Beijing Academy of Artificial Intelligence*-BAAI) . BAAI é uma organização privada sem fins lucrativos envolvida em pesquisa e desenvolvimento de IA.

In [ ]:
# Import das bibliotecas
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carrega os embeddings diretamente do HuggingFace
model_lm = HuggingFaceBgeEmbeddings(
    model_name=nome_modelo_bert,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

print("Tempo de carregamento do modelo LM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de carregamento do modelo LM:  0:00:13 (h:mm:ss)


In [ ]:
print(model_lm)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
) model_name='neuralmind/bert-large-portuguese-cased' cache_folder=None model_kwargs={'device': 'cuda'} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: '


Carrega o tokenizador do bert

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo_bert + ' da comunidade...')

tokenizer_bert = AutoTokenizer.from_pretrained(nome_modelo_bert)

Carregando o tokenizador neuralmind/bert-large-portuguese-cased da comunidade...


# 3 - Pergunta em texto longo

Artigos que auxiliaram a criar o recuperador de texto de perguntas em texto longo.

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

## 3.1 - Carrega os Documentos

Carrega de PDF ou HTML.

Escolha uma das formas de carregar os dados.

Existem outros formas de carregamento.

https://python.langchain.com/docs/modules/data_connection/document_loaders/

### 3.1.1 - Carrega os documentos de PDF

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf

####  Download do PDF


Download do PDF do livro "As Vítimas Algozes".

PDF: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977


Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070


In [ ]:
# Import das bibliotecas
# import subprocess

# # As Vítimas Algozes
# urlpdf = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'

# destino = 'arquivo1.pdf'

# # Executa o comando wget no prompt
# subprocess.call(["wget", urlpdf, "-O", destino])

#### Carrega o PDF



In [ ]:
# # Import das bibliotecas
# from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

# # Define o diretório
# diretorio = '/content'

# # Cria o carregar dos documentos do diretório
# # Pode ser usado o PyPDFLoader para um arquivo
# carregador = PyPDFDirectoryLoader(diretorio)

# # Carrega os documentos
# documentos = carregador.load()

### 3.1.2 - Carrega os documentos da WEB

https://python.langchain.com/docs/integrations/document_loaders/web_base

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

#### Carrega o HTML

Carrega o HTML do livro "As Vítimas Algozes".

HTML: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979

Biblioteca: https://www.literaturabrasileira.ufsc.br/documentos/?id=142070



In [ ]:
# Import das bibliotecas
from langchain.document_loaders import WebBaseLoader

urlhtml = "https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979"

# Cria o carregador da página
carregador = WebBaseLoader(urlhtml)

# Carrega os documentos
documentos = carregador.load()

Exibe parte dos dados carregados do PDF ou HTML.

In [ ]:
print("Quantidade de documentos(páginas):", len(documentos))
pagina = 0
print("Trecho página(", pagina, ") :", documentos[pagina].page_content[0:500])
print()
print("Medadados:", documentos[0].metadata)

Quantidade de documentos(páginas): 1
Trecho página( 0 ) : 















As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma ven

Medadados: {'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'title': 'As vítimas algozes - Joaquim Manuel de Macedo', 'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.'}


## 3.2 - Divide e sobrepõe os documentos em chuks

Carrega o documentos e realiza o divisão do documento em pedaços(chunks) e faz a sobreposição(overlap) para garantir o contexto semântico entre os pedaços.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto no link: https://chunkerizer.streamlit.app/

In [ ]:
# Import das bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import time

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

# Configura o divisor
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer_bert,
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    add_start_index = True, # Adiciona o índice de início do chunk no documento
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos documentos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(documentos)} documentos html em {tempo_final - tempo_inicio} segundos!")
print(f"Quantidade de chunks: {len(chunks)}")

Carregando e dividindo 1 documentos html em 3.3902273178100586 segundos!
Quantidade de chunks: 481


Mostra alguns chunks. A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)
    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    # print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  start_index: 16

As vítimas algozes - Joaquim Manuel de Macedo



Fonte: Biblioteca Digital de Literatura de Países Lusófonos

LITERATURA BRASILEIRA 
Textos literários em
meio eletrônico
As
Vítimas-Algozes, de Joaquim Manuel de Macedo
-----------------------------------------------------------------------
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  start_index: 235

Edição de base:
Biblioteca Nacional – setor de obras digitalizadas
ÍNDICE
SIMEÃO, O CRIOULO
PAI- RAIOL, O FEITICEIRO
LUCINDA, A
MUCAMA
CONCLUSÃO
I
SIMEÃO, O CRIOULO
I
No interior e principalmente longe da vila, ou da
freguesia e dos povoados há quase sempre uma venda perto da fazenda: é a
parasita que se apega à árvore; pior que isso, é a inimiga hipócrita que rende
vassalagem à sua vítima.
A venda de que falo

In [ ]:
maior_chunk_token = 0
maior_chunk_token_bert = 0
maior_chunk_character = 0

for i, chunk in enumerate(chunks):
    # Divide o chunk pelo espaço em branco
    tokens = chunk.page_content.split(" ")

    # Divide o chunk pelo tokenizador do BERT
    tokens_bert = tokenizer_bert.tokenize(chunk.page_content)

    # Documento HTML
    print('chunk #',i,' Doc:',chunk.metadata['source'],' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert), ' start_index:', chunk.metadata.get('start_index') )
    # Documento PDF
    #print('chunk #',i,' Doc:',chunk.metadata['source'],' Página:', chunk.metadata['page'], ' qtde char :', len(chunk.page_content),' qtde token :', len(tokens), ' qtde token bert:', len(tokens_bert),' start_index:', chunk.metadata.get('start_index') )

    # Procura os maiores valores
    if len(tokens) > maior_chunk_token:
      maior_chunk_token = len(tokens)
    if len(tokens_bert) > maior_chunk_token_bert:
      maior_chunk_token_bert = len(tokens_bert)
    if len(chunk.page_content) > maior_chunk_character:
      maior_chunk_character = len(chunk.page_content)

print("Maior chunk token:", maior_chunk_token)
print("Maior chunk token bert:", maior_chunk_token_bert)
print("Maior chunk character:", maior_chunk_character)

chunk # 0  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 217  qtde token : 25  qtde token bert: 50  start_index: 16
chunk # 1  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1429  qtde token : 220  qtde token bert: 361  start_index: 235
chunk # 2  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1530  qtde token : 253  qtde token bert: 394  start_index: 1357
chunk # 3  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1602  qtde token : 250  qtde token bert: 403  start_index: 2605
chunk # 4  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1623  qtde token : 244  qtde token bert: 406  start_index: 3902
chunk # 5  Doc: https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979  qtde char : 1552  qtde token : 237  qtde token ber

## 3.3 - Armazena os chunks

Utiliza o Chroma um banco de dados de embeddings de código aberto e leve.

https://www.trychroma.com/

https://heidloff.net/article/retrieval-augmented-generation-chroma-langchain/

In [ ]:
# Import das bibliotecas
from langchain.vectorstores import Chroma

# Armazena os chunks usando o Chroma
vectorstore = Chroma.from_documents(documents=chunks, embedding=model_lm, persist_directory="./bancodedados", )

## 3.4 - Função de envio de perguntas ao LLM

In [ ]:
# Import das bibliotecas
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
import torch
import gc

def avaliarContexto(texto):

  # Cria o texto de prompt
  prompt_template = """Você é um assistente de IA útil e fornece a resposta em língua portuguesa para a pergunta com base no contexto fornecido.
Contexto: {context}
>>PERGUNTA<< {question}
>>RESPOSTA<<"""

  # Cria o prompt
  # As variáveis "context" e "question" não podem ser alteradas
  # O parâmetro "context" recebe o contexto do carregado e armazenado no Chroma
  # O parâmetro "question" recebe a pergunta realizada sobre o contexto.
  prompt = PromptTemplate(input_variables=["context", "question"],
                          template=prompt_template)

  # Instancia o chain
  chain_type_kwargs = {"prompt": prompt}
  chain = RetrievalQA.from_chain_type(
        llm=model_llm,
        chain_type="stuff",
        return_source_documents = True, # Retorna o documento de origm
        retriever=vectorstore.as_retriever(), # Passa o contexto(livro carregado)
        #retriever=vectorstore.as_retriever(search_kwargs={'k':1}),
        chain_type_kwargs=chain_type_kwargs
    )

  # Executa o prompt no llm
  resultado = chain({"query":texto})

  # Esvazia a memória
  del chain
  gc.collect()
  torch.cuda.empty_cache()

  return resultado

## 3.5 - Algumas perguntas usando o contexto

Guarda o tempo de início de realização das perguntas

In [ ]:
# Importando as bibliotecas do Modelo
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

### Perguntas sobre o tempo

In [ ]:
texto = "Qual o período que ocorreu a história do texto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 A história do texto ocorreu no século XIX, pois é mencionado o "ministro da França no Rio de Janeiro".

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara inf

In [ ]:
texto = "Considerando o contexto em qual período que ocorreram os fatos?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os fatos ocorreram no século XIX, época em que a cidade do Rio de Janeiro era a capital do Brasil.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infunda

### Perguntas sobre personagens

In [ ]:
texto = "Qual o nome de todos os personagens do texto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os personagens são:

1. Lucinda
2. Cándida
3. Frederico
4. Paulo Borges (curandeiro)
5. Esméria (defunta)

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara 

In [ ]:
texto = "Considerando o contexto qual o nome de todos os personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os personagens são:

1. Esméria
2. Paulo Borges
3. Frederico
4. Cándida
5. Lucinda
6. Souvanel (que é na verdade Paulo 
Dermany)

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que 

In [ ]:
texto = "Quem são os principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Paulo Borges, Esméria, o curandeiro e a defunta (mencionada mas não identificada).

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido ad

In [ ]:
texto = "Quais os nomes dos principais personagens do contexto?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Paulo Borges, Esméria, o curandeiro e a defunta (mencionados como "ela").

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido adúltero su

In [ ]:
texto = "Considerando o contexto quem são os principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens da história são:

1. Lucinda - É a protagonista da história e a pessoa em questão.
2. Paulo B
orges - É o curandeiro que diagnosticou a morte de Lucinda como envenenamento.
3. Esméria - É a esposa de Paulo Borges e
 uma das suspeitas de ter envenenado Lucinda.
4. Frederico - É o marido de Lucinda e o autor do bilhete que acusa Souvan
el de estar envolvido na morte dela.
5. Cándida - É a mucama de Lucinda e que encontrou o bilhete de Frederico.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n–

In [ ]:
texto = "Considerando o contexto qual o nome dos principais personagens da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 Os principais personagens da história são:

1. Lucinda - a jovem mulher que está envolvida com o adivinho Paulo Borges 
e que acaba morrendo assassinada.
2. Paulo Borges - o adivinho responsável pela morte de Lucinda.
3. Esméria - a esposa 
de Paulo Borges, que é apresentada como uma pessoa frágil e vulnerável.
4. Frederico - o filho adotivo de Paulo Borges e
 Esméria, que está envolvido em uma relação amorosa com Cândida.
5. Cándida - a mucama de Paulo Borges e Esméria, que é 
apresentada como uma pessoa astuta e caridosa.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Que

In [ ]:
texto = "Qual o nome do pai da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 O nome do pai da Angêlica é Paulo Borges.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também

In [ ]:
texto = "Qual o nome do pai da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

 O nome do pai da Florinda é Paulo Borges.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também

In [ ]:
texto = "Qual o nome do pai do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 O nome do pai do Domingo é Simeão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da mãe da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Lucinda

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe era dirigida e que dizi

In [ ]:
texto = "Qual o nome da mãe da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A mãe da Florinda se chama Angélica.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a qu

In [ ]:
texto = "Qual o nome da mãe do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A mãe do Domingo se chama Eufêmia.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da mãe do Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Lucinda

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe era dirigida e que dizi

In [ ]:
texto = "Qual o nome da pessoa casada com a Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Frederico

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da pessoa casada com a Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 O nome da pessoa casada com a Florinda é Souvanel.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade l

In [ ]:
texto = "Qual o nome da pessoa casada com o Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A pessoa casada com o Domingo é chamada de Eufêmia.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome da pessoa casada com o Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A pessoa casada com o Simeão chama-se Esméria.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}


In [ ]:
texto = "Qual o nome dos filhos da Angêlica?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos filhos da Angêlica são: Frederico e Leonídia.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\x

In [ ]:
texto = "Qual o nome dos filhos da Florinda?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A resposta à pergunta é: LIBERTADO E FREDERICO.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu 

In [ ]:
texto = "Qual o nome dos filhos do Domingo?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos filhos do Domingo são: Cándida e Leonídia.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xadd

In [ ]:
texto = "Qual o nome dos filhos do Simeão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos filhos de Simeão são: Ana, Tiago, José, Simão e Judas.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\n

In [ ]:
texto = "A Angêlica é filha de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A Angêlica é filha de Lucinda.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido adúltero supôs enganar a Deus e aos homens, e\ntalvez 

In [ ]:
texto = "A Florinda é filha de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 A Florinda é filha de Esméria.

Origem: page_content='obedecendo ao primeiro impulso do amor alvoroçado, tinha marcado um novo\nencontro, em que provavelmente seria pela segunda vez escrava do amante algoz.\nEla o compreendeu, tremendo de vergonha e de medo, logo depois do desperto da\nconsciência; era porém tarde: o convite e o emprazamento já estavam dados, e\nLucinda desaparecera.\nQuando voltou ao quarto para vestir a senhora, a mucama\ndisse:\n– O pajem foi pôr o sinal no lugar ajustado\n– Que pressa! – observou Cândida.\n– Mas se é preciso aproveitar ocasiões, minha senhora!\nO dia estava como destinado para o recebimento de cartas\nimpor\xadtantes.\nCândida recebera o assustador bilhete de Souvanel ao\ndespertar de manhã.\nÀ tarde chegaram, vindas pelo correio, cartas de\nFrederico para Cândida e Leonídia.\nEnquanto sua mãe lia a carta do filho adotivo, que muito\nlongamente lhe escrevera, Cândida foi para o seu quarto e com desconfiança e\ncuriosi\xaddade leu também a que lhe 

In [ ]:
texto = "O Domingo é filho de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Não sou filho de ninguém, sou um assistente de IA.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido adúltero supôs enganar a Deus e ao

In [ ]:
Dtexto = "O Simeão é filho de quem?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 O Domingo é filho de Deus.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido adúltero supôs enganar a Deus e aos homens, e\ntalvez mes\

In [ ]:
texto = "Quais os nomes dos casais da história?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Os nomes dos casais são:

1. Esméria e Paulo Borges
2. Lucinda e Frederico

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.\nPaulo Borges tranqüilizou a revolta de seu ânimo, e\nconcentrou-se na dor da viuvez recente.\nEsméria ficou inocente a seus olhos, e quase que mais\nmereceu em compensação da suspeita que o curandeiro declarara infundada.\nO marido adúltero s

### Perguntas diversas

In [ ]:
texto = "Liste para mim as palavras que estão relacionadas com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Escravidão, oblações, sacrilégias, delírio, moribunda, fúnebre, suntuosa, honras, missas, ofício, setimo dia, pecador, 
incontrito, obstinado, imaginação, verdade, prevenções, ornamentos, forma, circunstâncias, acessórias, incidentais, inte
resse, lula, mater, dolor, viuvez, recentemente, exclusivamente, fonte, ornamentar, corruptos, pestíferos, contacto, inf
luência, fatal, bani, crime, sociedade, desmoralizar, empestar, assassinar, opressores.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus p

In [ ]:
texto = "Liste para mim a quantidade de  palavras que estão relacionadas com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras relacionadas com escravidão são:
escravo, escravidão, senhor, proprietário, escravidão, obediência, discipl
ina,
submissão, trabalho, força, condicionamento, rotina, materializado, indiferença,
estupidez, desmazelo, prejuízo, su
icídio, enfermeira, fuga, quilombo, seduzir,
perdura, resultado, riqueza, imoral, corrupção, refalsada, misterioso, dano
,
paciência, cálculo.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio

In [ ]:
texto = "Considerando o contexto que palavras possuem relação com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, as palavras "escravo" e "correção" estão presentes no texto e relacionadas à escravidão.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title': 'As vítimas algozes - Joaquim Manuel de Macedo'}

In [ ]:
texto = "Considerando o contexto que palavras possuem relação semântica com escravidão?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, as seguintes palavras têm relação semântica com escravidão:

* Simeão (escravo)
* Eufêmia (cúmplice)
* Barbudo (ou
tro cúmplice)
* Correção (prisão)
* Malvadeza (mau comportamento)
* Exceção (lei que permite a pena de morte)
* Justiça 
(lei que castiga os culpados)

 Essas palavras estão relacionadas à escravidão porque elas são usadas para descrever per
sonagens ou situações envolvidas na história de Simeão e da sua tragédia.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nel

In [ ]:
texto = "Considerando o contexto que palavras estão relacionadas a escravidão mas deixa isto implicito?"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 Sim, entendi. Com base neste contexto, podemos entender que Simeão é um escravo que tentou fugir da fazenda onde estava
 preso, mas foi capturado e castigado com a morte na forca. O Barbudo e Eufêmia são personagens que parecem estar envolv
idos na conspiração que levou à morte de Simeão. Aqui está a resposta em língua portuguesa para a sua pergunta, consider
ando o contexto fornecido.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte

Textos do trabalho do **Leandro da Silveira Dias**

In [ ]:
texto = "Considerando a palavra \"negro\" com alvo, identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras semânticamente similares a "negro" no contexto são:

* "falso" (para o personagem Souvanel)
* "roubou" (par
a o personagem Dermany)
* "falsificando" (para o personagem Dermany)

Estas palavras têm conotações negativas e podem se
r usadas para descrever o personagem ou a situação de uma maneira pejorativa.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes assim.

In [ ]:
texto = "Considerando a palavra alvo \"negro\", identifique outras palavras no contexto que são semanticamente similares."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras semânticamente similares à palavra "negro" no contexto são:

* "febre" (linha 10)
* "moribunda" (linha 12)

* "delírio" (linha 13)
* "envenenada" (linha 14)

 Essas palavras compartilham uma conotação negativa e associam-se a si
tuações adversas ou de sofrimento, como a doença e a morte da personagem Esméria.

Origem: page_content='O curandeiro turbou-se: de novo e com absurdo processo\nfez o exame do triste e enregelado corpo da vítima, e incapaz de compreender os\nsin\xadtomas que haviam escapado à sua ignorância, incapaz de apelar para os meios\ncientíficos que vingam a sociedade, reconhecendo no cadáver as provas\nirrecusáveis do crime do envenenador, o curandeiro charlatão, vaidoso do seu\ndiagnóstico, acabou por dizer com desfaçada impostura:\n– Envenenada!... Quem o disse, mentiu.\n– Está absolutamente certo disso?\n– Juro-o...\n– Que Deus perdoe a quem tal suspeitou! \n– Quem foi?\n– A defunta.\n– Delírio de moribunda: ela morreu da febre que eu\ndisse.\n– Antes as

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra \'negro\'"

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


   Nenhumas das palavras na sentença mencionada tem uma relação direta com a palavra "negro". A palavra "negro" não é me
ncionada naquela frase.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description': 'As vítimas algozes - Joaquim Manuel de Macedo', 'language': 'No language found.', 'source': 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', 'start_index': 107930, 'title'

In [ ]:
texto = " Na sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ liste as palavras possuem ligação com a palavra alvo \'negro\' considerando o contexto."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


  As palavras que possuem ligação com a palavra alvo "negro" no contexto são:

* Pundonor (referindo-se à honra e virtud
e que são características associadas à cultura afro-brasileira)
* Vergonha (associada à dignidade e self-respect, também
 valores culturais importantes na comunidade negra)
* Escravo (uma referência direta aos indivíduos negros que foram esc
ravizados e explorados durante a época colonial)
* Rancor (um termo que sugere a ideia de ressentimento e amargura, sent
imentos que podem ser relacionados à experiência histórica de opposição e discriminação sofrida pela população negra)
* 
Desejo da vingança (um tema que pode ser interpretado como uma forma de resistência e luta contra a opressão e a injusti
ça, especialmente na perspectiva da diáspora afro-brasileira).

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrí

In [ ]:
texto = "Usando leitura distante realize a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



A seguir estão as palavras relacionadas à palavra alvo 'negro' presentes na sentença:

1. Escravo - é evidente que esta
 palavra está diretamente relacionada com a raça negra.
2. Negro - embora não apareça explicitamente na sentença, podemo
s inferir que a referência à escravidão é devido ao fato de que a maioria das pessoas escravizadas eram negras.
3. Vergo
nha - Embora não seja uma palavra diretamente relacionada com a raça negra, podemos supor que a falta de vergonha mencio
nada na sentença se refere à forma como as pessoas negras foram tratadas e consideradas na época, o que poderia ter leva
do a uma sensação de falta de dignidade e honra.
4. Pundonor - novamente, embora não seja uma palavra diretamente relaci
onada com a raça negra, podemos supor que a ideia de pundonor mencionada na sentença se refere à forma como as pessoas n
egras foram tratadas e consideradas na época, o que poderia ter levado a uma sensação de falta de dignidade e honra.

Origem: page_content='si.\nSimeão c

In [ ]:
texto = "Considerando o contexto use leitura distante para realizar a analise da sentença \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ para encontrar e listar as palavras possuem ligação com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras relacionadas ao alvo "negro" presentes na frase são:

1. Negro (aparece duas vezes)
2. Escravo
3. Escravidã
o
4. Pundonor
5. Vergonha

As palavras "negro", "escravo" e "escravidão" estão diretamente relacionadas ao alvo "negro".
 Já as palavras "pundonor" e "vergonha" estão relacionadas ao tema da honra e da moralidade, respectivamente.

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espantosamente horrível não ficou impune. A lei\nvingou as ví\xadtimas.\nO Barbudo,\nEufêmia e outros cúmplices acham-se na casa da correção pagando sua malvadeza.\nSimeão subiu à forca; estrebuchou e morreu\ndebaixo dos pés do car\xadrasco.\nA lei de exceção matou o escravo e deixou com\nvida o Barbudo tão ce\xadlerado como ele, ou, se é possível, mais celerado que\nele.\nTudo isto é profundamente imoral e perverte a sociedade.' metadata={'description'

In [ ]:
texto = "Usando leitura distante realize a analise da sentença: \'Em falta de pundonor e de vergonha, que a escravidão não comporta, o escravo tem o rancor e o desejo da vingança.'\ e liste as palavras possuem relação semântica com a palavra alvo \'negro\'."

resultado = avaliarContexto(texto)

# Mostra o resultado
print_linhas_menores(resultado["result"])

# Verifique o documento de origem usado
print("\nOrigem:", resultado["source_documents"][0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 As palavras que possuem relação semântica com a palavra alvo "negro" na sentença são:

* Pundonor (porque o pundonor é 
uma qualidade que é associada a pessoas de coragem e honra, e a escravidão é frequentemente associada a negros)
* Vergon
ha (pois a vergonha é uma emoção que é associada a situações embarrassantes ou desonradoras, e a escravidão é vista como
 uma situação desonrosa para os negros)
* Rancor (pois o rancor é uma emoção que é associada a ressentimento e amargura,
 e a escravidão é uma situação que pode gerar esses sentimentos nos negros)
* Desejo da vingança (pois a vingança é uma 
emoção que é associada a um desejo de retaliação contra aqueles que foram considerados responsáveis por danos sofridos, 
e a escravidão é vista como uma injustiça que afeta principalmente os negros).

Origem: page_content='si.\nSimeão contara demais com as suas forças: ferido, e\ntendo perdido muito sangue, caiu desanimado, quando procurava saltar a cerca da\nfazenda.\nCONCLUSÃO\nO crime espa

Tempo final de execução das perguntas

In [ ]:
print("Tempo de execução das perguntas:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Tempo de execução das perguntas:  0:22:03 (h:mm:ss)
